In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow.keras
from tensorflow.keras.layers import *
from tensorflow.keras.models import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.linear_model import *
from sklearn.preprocessing import *

from sklearn.ensemble import *
from datetime import datetime
from tensorflow.keras.callbacks import *

In [2]:
df = pd.read_csv("./Datasets/Ro16/soccer-spi/spi_matches.csv")

In [3]:
df.head()

season        date  league_id                   league             team1  \
0    2016  2016-07-09       7921  FA Women's Super League   Liverpool Women   
1    2016  2016-07-10       7921  FA Women's Super League     Arsenal Women   
2    2016  2016-07-10       7921  FA Women's Super League  Chelsea FC Women   
3    2016  2016-07-16       7921  FA Women's Super League   Liverpool Women   
4    2016  2016-07-17       7921  FA Women's Super League  Chelsea FC Women   

                 team2   spi1   spi2   prob1   prob2  ...  importance1  \
0              Reading  51.56  50.42  0.4389  0.2767  ...          NaN   
1  Notts County Ladies  46.61  54.03  0.3572  0.3608  ...          NaN   
2      Birmingham City  59.85  54.64  0.4799  0.2487  ...          NaN   
3  Notts County Ladies  53.00  52.35  0.4289  0.2699  ...          NaN   
4        Arsenal Women  59.43  60.99  0.4124  0.3157  ...          NaN   

   importance2  score1  score2  xg1  xg2  nsxg1  nsxg2  adj_score1  adj_score2  
0          NaN     2.0     0.0  NaN  NaN    NaN    NaN         NaN         NaN  
1          NaN     2.0     0.0  NaN  NaN    NaN    NaN         NaN         NaN  
2          NaN     1.0     1.0  NaN  NaN    NaN    NaN         NaN         NaN  
3          NaN     0.0     0.0  NaN  NaN    NaN    NaN         NaN         NaN  
4          NaN     1.0     2.0  NaN  NaN    NaN    NaN         NaN         NaN  

[5 rows x 23 columns]

In [5]:
def dateconv (dates):
  date = datetime.strptime(dates,'%Y-%m-%d') 
  return date

df['date'] = df['date'].apply(lambda x : dateconv(x))

In [10]:
last_played = datetime(2021,2,15)
cl = df[(df['league']== "UEFA Champions League")]
all = df[(df['league']== "UEFA Champions League") | (df['league']=="Barclays Premier League") | (df['league']=="Italy Serie A") | (df['league']=="German Bundesliga") | (df['league']=="French Ligue 1")]
old = all[all['date']<last_played]
new = cl[cl['date']>last_played]

In [11]:
teams = pd.get_dummies(new[['team1','team2']])
teamsold = pd.get_dummies(old[['team1','team2']])

In [12]:
new = pd.concat([new,teams], axis=1)
old = pd.concat([old,teamsold], axis=1)

In [13]:
new.columns

Index(['season', 'date', 'league_id', 'league', 'team1', 'team2', 'spi1',
       'spi2', 'prob1', 'prob2', 'probtie', 'proj_score1', 'proj_score2',
       'importance1', 'importance2', 'score1', 'score2', 'xg1', 'xg2', 'nsxg1',
       'nsxg2', 'adj_score1', 'adj_score2', 'team1_Atalanta',
       'team1_Atletico Madrid', 'team1_Barcelona', 'team1_Bayern Munich',
       'team1_Borussia Dortmund', 'team1_Borussia Monchengladbach',
       'team1_Chelsea', 'team1_FC Porto', 'team1_Juventus', 'team1_Lazio',
       'team1_Liverpool', 'team1_Manchester City', 'team1_Paris Saint-Germain',
       'team1_RB Leipzig', 'team1_Real Madrid', 'team1_Sevilla FC',
       'team2_Atalanta', 'team2_Atletico Madrid', 'team2_Barcelona',
       'team2_Bayern Munich', 'team2_Borussia Dortmund',
       'team2_Borussia Monchengladbach', 'team2_Chelsea', 'team2_FC Porto',
       'team2_Juventus', 'team2_Lazio', 'team2_Liverpool',
       'team2_Manchester City', 'team2_Paris Saint-Germain',
       'team2_RB Leipzi

In [15]:
X_train = old[['season', 'spi1', 'spi2', 'score1','score2','prob1', 'prob2', 'probtie', 'proj_score1',
       'proj_score2','team1_Atalanta',
       'team1_Atletico Madrid', 'team1_Barcelona', 'team1_Bayern Munich',
       'team1_Borussia Dortmund', 'team1_Borussia Monchengladbach',
       'team1_Chelsea', 'team1_FC Porto', 'team1_Juventus', 'team1_Lazio',
       'team1_Liverpool', 'team1_Manchester City', 'team1_Paris Saint-Germain',
       'team1_RB Leipzig', 'team1_Real Madrid', 'team1_Sevilla FC',
       'team2_Atalanta', 'team2_Atletico Madrid', 'team2_Barcelona',
       'team2_Bayern Munich', 'team2_Borussia Dortmund',
       'team2_Borussia Monchengladbach', 'team2_Chelsea', 'team2_FC Porto',
       'team2_Juventus', 'team2_Lazio', 'team2_Liverpool',
       'team2_Manchester City', 'team2_Paris Saint-Germain',
       'team2_RB Leipzig', 'team2_Real Madrid', 'team2_Sevilla FC']]
X_test = new[['season', 'spi1', 'spi2','prob1', 'prob2', 'probtie', 'proj_score1',
       'proj_score2','team1_Atalanta',
       'team1_Atletico Madrid', 'team1_Barcelona', 'team1_Bayern Munich',
       'team1_Borussia Dortmund', 'team1_Borussia Monchengladbach',
       'team1_Chelsea', 'team1_FC Porto', 'team1_Juventus', 'team1_Lazio',
       'team1_Liverpool', 'team1_Manchester City', 'team1_Paris Saint-Germain',
       'team1_RB Leipzig', 'team1_Real Madrid', 'team1_Sevilla FC',
       'team2_Atalanta', 'team2_Atletico Madrid', 'team2_Barcelona',
       'team2_Bayern Munich', 'team2_Borussia Dortmund',
       'team2_Borussia Monchengladbach', 'team2_Chelsea', 'team2_FC Porto',
       'team2_Juventus', 'team2_Lazio', 'team2_Liverpool',
       'team2_Manchester City', 'team2_Paris Saint-Germain',
       'team2_RB Leipzig', 'team2_Real Madrid', 'team2_Sevilla FC']]

In [16]:
X_train.dropna(inplace=True)

In [17]:
X_test.dropna(inplace=True)

In [18]:
lm = LinearRegression()
lm.fit(X_train.drop(['score1','score2'],axis=1),X_train[['score1','score2']])
predictions  = lm.predict(X_test)
preds = []
for num in predictions:
  score1 = np.round(num[0])
  score2 = np.round(num[1])
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Lenny Score 1'] = score1
new['Lenny Score 2'] = score2
new.to_excel("ProjectionsCLRo16_LR.xlsx")

16
16


In [19]:
tX_train = X_train.drop(['score1','score2'],axis=1).values
tX_test =X_test.values
y_train = X_train[['score1','score2']].values

In [20]:
scaler = MinMaxScaler()
tX_train = scaler.fit_transform(tX_train)
tX_test = scaler.transform(tX_test)
tX_train.shape

(7158, 40)

In [21]:
def two_to_three(array):
    array = np.reshape(array, array.shape+(1,))
    return array

In [22]:
tX_test = two_to_three(tX_test)
tX_train = two_to_three(tX_train)
print(tX_test.shape)
print(tX_train.shape)

(16, 40, 1)
(7158, 40, 1)


In [23]:
model = Sequential()

In [24]:

model.add(SimpleRNN(46, return_sequences=True))
model.add(SimpleRNN(46, return_sequences=True))
model.add(SimpleRNN(46))
model.add(Dense(92, activation='relu'))
model.add(Dense(2))


In [25]:
model.compile(optimizer='adam',loss='mean_squared_error',metrics=['accuracy'])

In [26]:
early_stop = EarlyStopping(monitor='loss', mode='min', verbose=1, patience=25)

In [27]:
model.fit(tX_train,y_train,epochs=30)

Train on 7158 samples
Epoch 1/30
7158/7158 [==============================] - 14s 2ms/sample - loss: 1.5014 - accuracy: 0.6728
Epoch 2/30
7158/7158 [==============================] - 10s 1ms/sample - loss: 1.4320 - accuracy: 0.6887
Epoch 3/30
7158/7158 [==============================] - 11s 2ms/sample - loss: 1.4199 - accuracy: 0.6892
Epoch 4/30
7158/7158 [==============================] - 11s 2ms/sample - loss: 1.4146 - accuracy: 0.6964
Epoch 5/30
7158/7158 [==============================] - 13s 2ms/sample - loss: 1.4127 - accuracy: 0.6940
Epoch 6/30
7158/7158 [==============================] - 17s 2ms/sample - loss: 1.4143 - accuracy: 0.6921
Epoch 7/30
7158/7158 [==============================] - 16s 2ms/sample - loss: 1.4064 - accuracy: 0.6971
Epoch 8/30
7158/7158 [==============================] - 14s 2ms/sample - loss: 1.4008 - accuracy: 0.6967
Epoch 9/30
7158/7158 [==============================] - 15s 2ms/sample - loss: 1.3961 - accuracy: 0.7001
Epoch 10/30
7158/7158 [==========

In [28]:
predictions  = model.predict(tX_test)
preds = []
for num in predictions:
  score1 = np.round(num[0]) if num[0]>=0 else 0
  score2 = np.round(num[1]) if num[1]>=0 else 0
  preds.append((score1,score2))

len(preds)

score1 = []
score2 = []

for num in preds:
  score1.append(num[0])
  score2.append(num[1])

print(len(score1))
print(len(score2))

new['Tanner Score 1'] = score1
new['Tanner Score 2'] = score2
new.to_excel("ProjectionsCLRo16_TF.xlsx")

16
16
